### Thesis notebook 4.3. - NOVA IMS

#### LSTM - Temporal data representation

In this notebook, we will finally start our application of temporal representation using LSTMs and bi-directional LSTMs.
The argument for the usage of Deep Learning stems from the fact that sequences themselves encode information that can be extracted using Recurrent Neural Networks and, more specifically, Long Short Term Memory Units.

#### First Step: Setup a PyTorch environment that enables the use of GPU for training. 

The following cell wll confirm that the GPU will be the default device to use.

In [1]:
import torch
import pycuda.driver as cuda

cuda.init()
## Get Id of default device
torch.cuda.current_device()
# 0
cuda.Device(0).name() # '0' is the id of your GPU

#set all tensors to gpu
torch.set_default_tensor_type('torch.cuda.FloatTensor')

#### Second Step: Import the relevant packages and declare global variables

In [2]:
#import necessary modules/libraries
import numpy as np
import scipy
import pandas as pd
import datetime as dt
import warnings
import time

#tqdm to monitor progress
from tqdm.notebook import tqdm, trange
tqdm.pandas(desc="Progress")

#time related features
from datetime import timedelta
from copy import copy, deepcopy

#vizualization
import matplotlib.pyplot as plt
import seaborn as sns

#imblearn, scalers, kfold and metrics
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer,PowerTransformer
from sklearn.model_selection import train_test_split, RepeatedKFold, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, recall_score, classification_report, average_precision_score, precision_recall_curve

#import torch related
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable 
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler


#and optimizer of learning rate
from torch.optim.lr_scheduler import ReduceLROnPlateau

#import pytorch modules
warnings.filterwarnings('ignore')

In [3]:
#global variables that may come in handy
#course threshold sets the % duration that will be considered (1 = 100%)
duration_threshold = [0.1, 0.25, 0.33, 0.5, 1]

#colors for vizualizations
nova_ims_colors = ['#BFD72F', '#5C666C']

#standard color for student aggregates
student_color = '#474838'

#standard color for course aggragates
course_color = '#1B3D2F'

#standard continuous colormap
standard_cmap = 'viridis_r'

#Function designed to deal with multiindex and flatten it
def flattenHierarchicalCol(col,sep = '_'):
    '''converts multiindex columns into single index columns while retaining the hierarchical components'''
    if not type(col) is tuple:
        return col
    else:
        new_col = ''
        for leveli,level in enumerate(col):
            if not level == '':
                if not leveli == 0:
                    new_col += sep
                new_col += level
        return new_col
    
#number of replicas - number of repeats of stratified k fold - in this case 10
replicas = 30

#names to display on result figures
date_names = {
             'Date_threshold_10': '10% of Course Duration',   
             'Date_threshold_25': '25% of Course Duration', 
             'Date_threshold_33': '33% of Course Duration', 
             'Date_threshold_50': '50% of Course Duration', 
             'Date_threshold_100':'100% of Course Duration', 
            }

target_names = {
                'exam_fail' : 'At risk - Exam Grade',
                'final_fail' : 'At risk - Final Grade', 
                'exam_gifted' : 'High performer - Exam Grade', 
                'final_gifted': 'High performer - Final Grade'
                }

#targets
targets = ['exam_fail' , 'final_fail' , 'exam_gifted' , 'final_gifted']
temporal_columns = ['0 to 4%', '4 to 8%', '8 to 12%', '12 to 16%', '16 to 20%', '20 to 24%',
       '24 to 28%', '28 to 32%', '32 to 36%', '36 to 40%', '40 to 44%',
       '44 to 48%', '48 to 52%', '52 to 56%', '56 to 60%', '60 to 64%',
       '64 to 68%', '68 to 72%', '72 to 76%', '76 to 80%', '80 to 84%',
       '84 to 88%', '88 to 92%', '92 to 96%', '96 to 100%']

#### Step 3: Import data and take a preliminary look at it 

In [4]:
#imports dataframes
course_programs = pd.read_excel("../Data/Modeling Stage/Nova_IMS_Temporal_Datasets_25_splits.xlsx", 
                                dtype = {
                                    'course_encoding' : int,
                                    'userid' : int},
                               sheet_name = None)

#save tables 
student_list = pd.read_csv('../Data/Modeling Stage/Nova_IMS_Filtered_targets.csv', 
                         dtype = {
                                   'course_encoding': int,
                                   'userid' : int,
                                   })

#drop unnamed 0 column
for i in course_programs:
        
    #merge with the targets we calculated on the other 
    course_programs[i] = course_programs[i].merge(student_list, on = ['course_encoding', 'userid'], how = 'inner')
    course_programs[i].drop(['Unnamed: 0', 'exam_mark', 'final_mark'], axis = 1, inplace = True)
    
    #convert results to object
    course_programs[i]['course_encoding'], course_programs[i]['userid'] = course_programs[i]['course_encoding'].astype(object), course_programs[i]['userid'].astype(object)

In [5]:
course_programs['Date_threshold_100'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9296 entries, 0 to 9295
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   course_encoding  9296 non-null   object
 1   userid           9296 non-null   object
 2   0 to 4%          9296 non-null   int64 
 3   4 to 8%          9296 non-null   int64 
 4   8 to 12%         9296 non-null   int64 
 5   12 to 16%        9296 non-null   int64 
 6   16 to 20%        9296 non-null   int64 
 7   20 to 24%        9296 non-null   int64 
 8   24 to 28%        9296 non-null   int64 
 9   28 to 32%        9296 non-null   int64 
 10  32 to 36%        9296 non-null   int64 
 11  36 to 40%        9296 non-null   int64 
 12  40 to 44%        9296 non-null   int64 
 13  44 to 48%        9296 non-null   int64 
 14  48 to 52%        9296 non-null   int64 
 15  52 to 56%        9296 non-null   int64 
 16  56 to 60%        9296 non-null   int64 
 17  60 to 64%        9296 non-null   

In [6]:
course_programs['Date_threshold_100'].describe(include = 'all')

,course_encoding,userid,0 to 4%,4 to 8%,8 to 12%,12 to 16%,16 to 20%,20 to 24%,24 to 28%,28 to 32%,...,76 to 80%,80 to 84%,84 to 88%,88 to 92%,92 to 96%,96 to 100%,exam_fail,final_fail,exam_gifted,final_gifted
count,9296.0,9296.0,9296.000000,9296.000000,9296.000000,9296.000000,9296.000000,9296.000000,9296.000000,9296.000000,...,9296.000000,9296.000000,9296.000000,9296.000000,9296.000000,9296.0,9296.000000,9296.000000,9296.000000,9296.000000
unique,138.0,1590.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,150.0,3178.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,178.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,1.081863,8.307874,10.752797,11.193739,10.127797,8.966652,10.545396,11.445245,...,11.718051,13.136403,22.827883,27.341007,12.599613,0.0,0.201377,0.149957,0.276893,0.308090
std,NaN,NaN,3.526351,13.580025,13.626754,16.400023,14.291254,12.180177,13.507892,15.932226,...,28.186874,36.690068,47.158607,54.963959,35.194597,0.0,0.401051,0.357048,0.447487,0.461729
min,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,0.000000,0.000000,1.000000,2.000000,2.000000,1.000000,2.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,0.000000,2.000000,7.000000,7.000000,6.000000,5.000000,7.000000,7.000000,...,2.000000,2.000000,4.000000,2.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
75%,NaN,NaN,1.000000,12.000000,15.000000,15.000000,13.000000,13.000000,14.000000,14.000000,...,10.000000,10.000000,23.000000,27.000000,5.000000,0.0,0.000000,0.000000,1.000000,1.000000


In our first attempt, we will use the absolute number of clicks made by each student - scaled using standard scaler. 
Therefore, we can start by immediately placing our course encoding/userid pairings into the index.

In [7]:
#function for scalers
def normalize(dataset,scaler):
    
    if scaler == 'MinMax':
        pt = MinMaxScaler()
    elif scaler == 'Standard':
        pt = StandardScaler()
    elif scaler == 'Robust':
        pt = RobustScaler()
    elif scaler == 'Quantile':
        pt = QuantileTransformer()
    else:
        pt = PowerTransformer(method='yeo-johnson')
    
    data = pt.fit_transform(dataset)
    
    # convert the array back to a dataframe
    normalized_df = pd.DataFrame(data,columns=dataset.columns)
    return normalized_df  

In [8]:
#create backup
normalized_data = deepcopy(course_programs)

#convert index
for i in tqdm(normalized_data):
    normalized_data[i].set_index(['course_encoding', 'userid'], drop = True, inplace = True)
    normalized_data[i].fillna(0, inplace = True)
    #Then, apply normalize function to rescale the train columns
    normalized_data[i] = normalize(normalized_data[i].filter(temporal_columns),'Standard')
    
    #and remerge target columns
    normalized_data[i][targets] =  deepcopy(course_programs[i][targets])

  0%|          | 0/5 [00:00<?, ?it/s]

#### Implementing Cross-Validation with Deep Learning Model

**1. Create the Deep Learning Model**

In this instance, we will follow-up with on the approach used in Chen & Cui - CrossEntropyLoss with applied over a softmax layer.

In [9]:
class LSTM_Uni(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM_Uni, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first = True) #lstm
        
        self.dropout = nn.Dropout(p = 0.5)
    
        self.fc = nn.Linear(self.hidden_size, num_classes) #fully connected last layer

    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        
        #Xavier_init for both H_0 and C_0
        torch.nn.init.xavier_normal_(h_0)
        torch.nn.init.xavier_normal_(c_0)
        
        # Propagate input through LSTM
        lstm_out, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        last_output = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        
        #we are interested in only keeping the last output
        drop_out = self.dropout(last_output)
        pre_softmax = self.fc(drop_out) #Final Output - dense
        return pre_softmax

**2. Define the train and validation Functions**

In [10]:
def train_epoch(model,dataloader,loss_fn,optimizer):
    
    train_loss,train_correct=0.0,0 
    model.train()
    for X, labels in dataloader:

        optimizer.zero_grad()
        output = model(X)
        loss = loss_fn(output,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X.size(0)
        scores, predictions = torch.max(F.log_softmax(output.data), 1)
        train_correct += (predictions == labels).sum().item()
        
    return train_loss,train_correct
  
def valid_epoch(model,dataloader,loss_fn):
    valid_loss, val_correct = 0.0, 0
    targets = []
    y_pred = []
    probability_1 = []
    
    model.eval()
    for X, labels in dataloader:

        output = model(X)
        loss=loss_fn(output,labels)
        valid_loss+=loss.item()*X.size(0)
        probability_1.append(F.softmax(output.data)[:,1])
        predictions = torch.argmax(output, dim=1)
        val_correct+=(predictions == labels).sum().item()
        targets.append(labels)
        y_pred.append(predictions)
    
    #concat all results
    targets = torch.cat(targets).data.cpu().numpy()
    y_pred = torch.cat(y_pred).data.cpu().numpy()
    probability_1 = torch.cat(probability_1).data.cpu().numpy()
    
    #calculate precision, recall and AUC score
    
    precision = precision_score(targets, y_pred)
    recall = recall_score(targets, y_pred)
    auroc = roc_auc_score(targets, probability_1)
    
    #return all
    return valid_loss,val_correct, precision, recall, auroc

**3. Define main hyperparameters of the model, including splits**

In [11]:
#Model
num_epochs = 100 #50 epochs
learning_rate = 0.01 #0.01 lr
input_size = 1 #number of features
hidden_size = 40 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

#Shape of Output as required for SoftMax Classifier
num_classes = 2 #output shape

batch_size = 32

k=10
splits= RepeatedKFold(n_splits=k, n_repeats=replicas, random_state=15) #kfold of 10 with 30 replicas
criterion = nn.CrossEntropyLoss()    # cross-entropy for classification

**4. Make the splits and Start Training**

In [ ]:
for i in tqdm(normalized_data.keys()):
    
    print(i)
    threshold_dict = {} #dict to store information in for each threshold
    data = deepcopy(normalized_data[i])
    
    #set X and Y columns
    X = data[data.columns[:25]] #different timesteps
    y = data[data.columns[-4:]] #the 4 different putative targets
    
    for k in tqdm(targets):
        print(k)
        
        #Start with train test split
        X_train_val, X_test, y_train_val, y_test, = train_test_split(
                                    X,
                                   y[k], #replace when going for multi-target 
                                   test_size = 0.20,
                                   random_state = 15,
                                   shuffle=True,
                                   stratify = y[k] #replace when going for multi-target
                                    )

        # ##second, convert everything to pytorch tensor - we will convert to tensor dataset and 
        X_train_tensors = Variable(torch.Tensor(X_train_val.values))
        X_test_tensors = Variable(torch.Tensor(X_test.values))

        y_train_tensors = Variable(torch.Tensor(y_train_val.values))
        y_test_tensors = Variable(torch.Tensor(y_test.values)) 

        #now, we have a dataset with features
        print('Before reshaping:')
        print("Training Shape", X_train_tensors.shape, y_train_tensors.shape)
        print("Testing Shape", X_test_tensors.shape, y_test_tensors.shape)

        #reshaping to rows, timestamps, features 
        X_train_tensors = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], X_train_tensors.shape[1], 1))
        X_test_tensors = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], X_test_tensors.shape[1], 1))

        #repeat for y
        y_train_tensors = y_train_tensors.type(torch.cuda.LongTensor)
        y_test_tensors = y_test_tensors.type(torch.cuda.LongTensor)

        print('\nAfter reshaping:')
        print("Training Shape", X_train_tensors.shape, y_train_tensors.shape)
        print("Testing Shape", X_test_tensors.shape, y_test_tensors.shape)

        #create dataset
        dataset = TensorDataset(X_train_tensors, y_train_tensors)
        
        #create dict to store fold performance
        foldperf={}
        
        #reset "best accuracy for treshold i and target k"
        
        best_accuracy = 0

        for fold, (train_idx,val_idx) in tqdm(enumerate(splits.split(np.arange(len(dataset))))):

            print('Split {}'.format(fold + 1))

            train_sampler = SubsetRandomSampler(train_idx)
            val_sampler = SubsetRandomSampler(val_idx)
            train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
            val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)
    
            #creates new model for each 
            model = LSTM_Uni(num_classes, input_size, hidden_size, num_layers, X_train_tensors.shape[1]).to('cuda') #our lstm class
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
            scheduler = ReduceLROnPlateau(optimizer, 
                                  'min', 
                                  patience = 10,
                                  cooldown = 20,
                                 verbose = True)
    
            history = {'train_loss': [], 'val_loss': [],'train_acc':[],'val_acc':[], 'precision': [],
                      'recall' : [], 'auroc': []}

            for epoch in tqdm(range(num_epochs)):
                train_loss, train_correct=train_epoch(model,train_loader,criterion,optimizer)
                val_loss, val_correct, precision, recall, auroc = valid_epoch(model,val_loader,criterion)

                train_loss = train_loss / len(train_loader.sampler)
                train_acc = train_correct / len(train_loader.sampler) * 100
                val_loss = val_loss / len(val_loader.sampler)
                val_acc = val_correct / len(val_loader.sampler) * 100
        
        
                if (epoch+1) % 10 == 0: 
                    print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Validation Loss:{:.3f} AVG Training Acc {:.2f} % AVG Validation Acc {:.2f} %".format(epoch + 1,
                                                                                                             num_epochs,
                                                                                                             train_loss,
                                                                                                             val_loss,
                                                                                                             train_acc,
                                                                                                             val_acc))
                history['train_loss'].append(train_loss)
                history['val_loss'].append(val_loss)
                history['train_acc'].append(train_acc)
                history['val_acc'].append(val_acc)
                history['precision'].append(precision)
                history['recall'].append(recall)
                history['auroc'].append(auroc)
                scheduler.step(val_loss)
    
                if val_acc > best_accuracy:
            
                #replace best accuracy and save best model
                    print(f'New Best Accuracy found: {val_acc:.2f}%\nEpoch: {epoch + 1}')
                    best_accuracy = val_acc
                    best = deepcopy(model)
                    curr_epoch = epoch + 1
                    
            #store fold performance
            foldperf['fold{}'.format(fold+1)] = history
        
        #saves fold performance for target 
        threshold_dict[k] = pd.DataFrame.from_dict(foldperf, orient='index') # convert dict to dataframe
        
        #explode to get eacxh epoch as a row
        threshold_dict[k] = threshold_dict[k].explode(list(threshold_dict[k].columns))
        torch.save(best,f"../Models/{i}/Nova_IMS_best_{k}_{curr_epoch}_epochs.h")
        
    # from pandas.io.parsers import ExcelWriter
    with pd.ExcelWriter(f"../Data/Modeling Stage/Results/IMS/Clicks per % duration/25_splits_{i}_{replicas}_replicas.xlsx") as writer:  
        for sheet in targets:
                threshold_dict[sheet].to_excel(writer, sheet_name=str(sheet))

  0%|          | 0/5 [00:00<?, ?it/s]

Date_threshold_10


  0%|          | 0/4 [00:00<?, ?it/s]

exam_fail
Before reshaping:
Training Shape torch.Size([7436, 25]) torch.Size([7436])
Testing Shape torch.Size([1860, 25]) torch.Size([1860])

After reshaping:
Training Shape torch.Size([7436, 25, 1]) torch.Size([7436])
Testing Shape torch.Size([1860, 25, 1]) torch.Size([1860])


0it [00:00, ?it/s]

...

Epoch:10/100 AVG Training Loss:0.410 AVG Validation Loss:0.414 AVG Training Acc 85.07 % AVG Validation Acc 84.41 %
Epoch:20/100 AVG Training Loss:0.407 AVG Validation Loss:0.411 AVG Training Acc 85.03 % AVG Validation Acc 84.41 %
Epoch:30/100 AVG Training Loss:0.398 AVG Validation Loss:0.406 AVG Training Acc 85.12 % AVG Validation Acc 84.41 %
Epoch:40/100 AVG Training Loss:0.392 AVG Validation Loss:0.413 AVG Training Acc 85.10 % AVG Validation Acc 84.41 %
Epoch:50/100 AVG Training Loss:0.383 AVG Validation Loss:0.410 AVG Training Acc 85.27 % AVG Validation Acc 84.68 %
Epoch    52: reducing learning rate of group 0 to 1.0000e-03.
Epoch:60/100 AVG Training Loss:0.353 AVG Validation Loss:0.423 AVG Training Acc 85.48 % AVG Validation Acc 84.27 %
Epoch:70/100 AVG Training Loss:0.341 AVG Validation Loss:0.444 AVG Training Acc 85.80 % AVG Validation Acc 83.87 %
Epoch:80/100 AVG Training Loss:0.327 AVG Validation Loss:0.455 AVG Training Acc 86.45 % AVG Validation Acc 84.14 %
Epoch    83: reduc

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch:10/100 AVG Training Loss:0.410 AVG Validation Loss:0.430 AVG Training Acc 85.06 % AVG Validation Acc 84.52 %
Epoch:20/100 AVG Training Loss:0.407 AVG Validation Loss:0.410 AVG Training Acc 85.06 % AVG Validation Acc 84.52 %
Epoch:30/100 AVG Training Loss:0.400 AVG Validation Loss:0.401 AVG Training Acc 85.01 % AVG Validation Acc 84.52 %
Epoch    37: reducing learning rate of group 0 to 1.0000e-03.
Epoch:40/100 AVG Training Loss:0.384 AVG Validation Loss:0.402 AVG Training Acc 85.07 % AVG Validation Acc 84.52 %
Epoch:50/100 AVG Training Loss:0.371 AVG Validation Loss:0.405 AVG Training Acc 85.34 % AVG Validation Acc 84.39 %
Epoch:60/100 AVG Training Loss:0.359 AVG Validation Loss:0.414 AVG Training Acc 85.45 % AVG Validation Acc 83.98 %
Epoch    68: reducing learning rate of group 0 to 1.0000e-04.
Epoch:70/100 AVG Training Loss:0.347 AVG Validation Loss:0.420 AVG Training Acc 85.55 % AVG Validation Acc 84.12 %
Epoch:80/100 AVG Training Loss:0.343 AVG Validation Loss:0.423 AVG Trai

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch:10/100 AVG Training Loss:0.406 AVG Validation Loss:0.430 AVG Training Acc 85.10 % AVG Validation Acc 84.12 %
Epoch:20/100 AVG Training Loss:0.402 AVG Validation Loss:0.440 AVG Training Acc 85.10 % AVG Validation Acc 84.12 %
